In [2]:
import numpy as np
import numpy
numpy.set_printoptions(threshold=numpy.nan)

In [3]:
import scipy.io
mat = scipy.io.loadmat('DadosLista4.mat')

In [4]:
dados = np.array(mat['Dados'])
y = np.array(mat['y'])

In [5]:
sizeDados = len(dados)
dados1, dados2, dados3 = dados[ : int((sizeDados/3))] , dados[int((sizeDados/3)) : int((2*sizeDados/3)) ], dados[ int((2*sizeDados/3)) : ]
y1, y2, y3 = y[ : int((sizeDados/3))] , y[int((sizeDados/3)) : int((2*sizeDados/3)) ], y[ int((2*sizeDados/3)) : ]

In [6]:
#Classe 1 treino
randomTreino1 = np.random.choice(dados1.shape[0], 200, replace=False)
dadosTreino1 = dados1[randomTreino1]
yTreino1 = y1[randomTreino1]

#Classe 1 treino
randomTreino2 = np.random.choice(dados2.shape[0], 200, replace=False)
dadosTreino2 = dados2[randomTreino2]
yTreino2 = y2[randomTreino2]

#Classe 1 treino
randomTreino3 = np.random.choice(dados3.shape[0], 200, replace=False)
dadosTreino3 = dados3[randomTreino3]
yTreino3 = y3[randomTreino3]



In [7]:
#Classe 1 teste
a1_rows = set(map(tuple, dados1))
a2_rows = set(map(tuple, dadosTreino1))
dadosTeste1 = np.array(list(a1_rows.difference(a2_rows)))
yTeste1 = np.ones((300,1))

#Classe 2 teste
a1_rows = set(map(tuple, dados2))
a2_rows = set(map(tuple, dadosTreino2))
dadosTeste2 = np.array(list(a1_rows.difference(a2_rows)))
yTeste2 = np.full((300,1), 2)

#Classe 3 teste
a1_rows = set(map(tuple, dados3))
a2_rows = set(map(tuple, dadosTreino3))
dadosTeste3 = np.array(list(a1_rows.difference(a2_rows)))
yTeste3 = np.full( (300,1), 3)


In [8]:
dadosTreino = np.concatenate((dadosTreino1, dadosTreino2, dadosTreino3), axis=0)
len(dadosTreino)

600

In [9]:
dadosTeste = np.concatenate((dadosTeste1, dadosTeste2, dadosTeste3), axis=0)
len(dadosTeste)

900

In [10]:
yTreino = np.concatenate((yTreino1, yTreino2, yTreino3), axis=0 )
len(yTreino)

600

In [11]:
yTeste = np.concatenate( (yTeste1, yTeste2, yTeste3), axis=0)
len(yTeste)

900

In [39]:
#calcula mi
miG = dadosTreino.mean(axis=0)
mi1 = dadosTreino1.mean(axis=0)
mi2 = dadosTreino2.mean(axis=0)
mi3 = dadosTreino3.mean(axis=0)

In [61]:
#somatorio
n1 = dadosTreino1.shape[0]
n2 = dadosTreino2.shape[0]
n3 = dadosTreino3.shape[0]

print(dadosTreino1[0])
print(mi1)
print(dadosTreino1[0] - mi1)



#sum1 = [(dadosTreino1[linha] - mi1).dot( (dadosTreino1[linha] - mi1).T ) for linha in range(n1)]
aux = (dadosTreino1[0] - mi1)
sum1 = [ np.reshape(aux, (1,2)).T.dot(np.reshape(aux, (1,2))) for linha in range(n1)]

sum1 = np.sum(np.array(sum1), axis=0)
print(sum1)

ec1 = sum1 / (n1 - 1)
print(ec1.shape)

#sum2 = [sum((dadosTreino2[linha] -mi2).dot( (dadosTreino2[linha] - mi2).T )) for linha in range(n2)]
aux = (dadosTreino2[0] - mi2)
sum2 = [ np.reshape(aux, (1,2)).T.dot(np.reshape(aux, (1,2))) for linha in range(n2)]

sum2 = np.sum(np.array(sum2), axis=0)

ec2 = np.array(sum2) / (n2 - 1)

#sum3 = [sum((dadosTreino3[linha] -mi3).dot( (dadosTreino3[linha] - mi3).T )) for linha in range(n3)]
aux = (dadosTreino3[0] - mi3)
sum3 = [ np.reshape(aux, (1,2)).T.dot(np.reshape(aux, (1,2))) for linha in range(n3)]

sum3 = np.sum(np.array(sum3), axis=0)

ec3 = np.array(sum3) / (n3 - 1)

[ 0.51511652  0.31993237]
[ 0.70619913  0.50677432]
[-0.19108262 -0.18684195]
[[ 7.30251325  7.14044978]
 [ 7.14044978  6.98198295]]
(2, 2)


TypeError: 'numpy.float64' object is not iterable

In [40]:
#Probabilidade de x para a classe C referente ao mi e ec(sigma) => P(x|c)
def prob(mi, ec, x):
    np.exp( (-1/2) * (x - mi).T * np.linalg.inv(ec) * (x - mi)  )